In [2]:
# Backtesting a Trading Strategy using Object Oriented Programming [OOP]
# https://www.youtube.com/watch?v=yTupVd6D9m8

In [2]:
import yfinance as yf
# data handling
import pandas as pd
# calculation purporses
import numpy as np
# technical indictor
import ta
# for visualization
import matplotlib.pyplot as plt

In [3]:
class Backtest:
    def __init__(self, symbol):
        self.symbol = symbol
        self.df = yf.download(self.symbol, start='2019-01-01')
        
        if self.df.empty:
            print("No Data Pulled")
        else:
            self.calc_indicators()
            self.generate_signals()
            self.loop_it()
            
    def calc_indicators(self):
        self.df['ma_20'] = self.df.Close.rolling(20).mean()
        # rolling standard deviation
        self.df['vol'] = self.df.Close.rolling(20).std()
        self.df['upper_bb'] = self.df.ma_20 + (2*self.df.vol)
        self.df['lower_bb'] = self.df.ma_20 - (2*self.df.vol)
        self.df['rsi'] = ta.momentum.rsi(self.df.Close, window=6)
        self.df.dropna(inplace=True)
        
    def generate_signals(self):
        conditions = [(self.df.rsi < 30) & (self.df.Close < self.df.lower_bb),
              (self.df.rsi > 70) & (self.df.Close < self.df.upper_bb)]
        choices = ['Buy', 'Sell']
        self.df['signal'] = np.select(conditions, choices)
        # close row before == shifted_close --> makes it more convenient to access signal when Backtester is iterating
        self.df['shifted_Close'] = self.df.Close.shift()
        self.df.dropna(inplace=True)
        
    def loop_it(self):
        position = False
        buydates, selldates = [], []
        
        for index, row in self.df.iterrows():
            if not position and row['signal'] == 'Buy':
                buydates.append(index)
                position = True
            if position:
                if row['signal'] == 'Sell': # or row.shifted_Close < 0.95 * buyprices[-1]:
                    selldates.append(index)
                    position = False
                    
        self.buy_arr = self.df.loc[buydates].Open
        self.sell_arr = self.df.loc[selldates].Open


In [4]:
instance = Backtest('SPY')
instance.df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ma_20,vol,upper_bb,lower_bb,rsi,signal,shifted_Close
Date,,,,,,,,,,,,,
2019-01-31,267.510010,270.470001,267.269989,269.929993,254.740982,104012100,260.600499,5.868911,272.338321,248.862677,77.396448,Sell,267.579987
2019-02-01,270.149994,271.200012,269.179993,270.059998,254.863724,85782500,261.892999,4.822431,271.537860,252.248137,77.679490,Sell,269.929993
2019-02-04,270.109985,272.029999,269.359985,271.959991,256.656769,60744800,262.871498,4.777960,272.427418,253.315579,81.698620,Sell,270.059998
2019-02-05,272.440002,273.440002,271.880005,273.100006,257.732574,79552800,263.807498,4.859847,273.527191,254.087805,83.799048,Sell,271.959991
2019-02-06,272.790009,273.339996,271.920013,272.739990,257.392883,58347800,264.605998,4.953759,274.513517,254.698479,80.306309,Sell,273.100006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,390.429993,398.589996,390.200012,397.779999,397.779999,70964200,411.986502,13.526727,439.039956,384.933048,41.717629,0,390.760010
2022-09-08,395.390015,400.859985,394.119995,400.380005,400.380005,80821700,411.006003,13.626410,438.258822,383.753183,48.166345,0,397.779999
2022-09-09,402.739990,407.510010,402.459991,406.600006,406.600006,76618100,410.336504,13.490031,437.316567,383.356441,60.661709,0,400.380005


In [6]:
instance.buy_arry

AttributeError: 'Backtest' object has no attribute 'buy_arry'